In [ ]:
import torch
import numpy as np
import torchvision

In [ ]:
cf_image = torch.load('../results/shapes_3d/closedform/latent_traversal_eps3/result_images3.pth')
cf_image_ours = torch.load('../results/shapes_3d/closedform_ours/latent_traversal_params_eps3/result_images3.pth')
cf_image_sample_two = torch.load('../results/shapes_3d/closedform/latent_traversal_eps3/result_images5.pth')
cf_image_ours_sample_two = torch.load('../results/shapes_3d/closedform_ours/latent_traversal_params_eps3/result_images5.pth')

cf_image_wall = torch.load('../results/shapes_3d/closedform/latent_traversal_eps3/result_images43.pth')
cf_image_ours_wall = torch.load('../results/shapes_3d/closedform_ours/latent_traversal_params_eps3/result_images43.pth')
cf_image_sample_two_wall = torch.load('../results/shapes_3d/closedform/latent_traversal_eps3/result_images72.pth')
cf_image_ours_sample_two_wall = torch.load('../results/shapes_3d/closedform_ours/latent_traversal_params_eps3/result_images72.pth')

cf_image_object = torch.load('../results/shapes_3d/closedform/latent_traversal_eps3/result_images68.pth')
cf_image_ours_object = torch.load('../results/shapes_3d/closedform_ours/latent_traversal_params_eps3/result_images68.pth')
cf_image_sample_two_object = torch.load('../results/shapes_3d/closedform/latent_traversal_eps3/result_images7.pth')
cf_image_ours_sample_two_object = torch.load('../results/shapes_3d/closedform_ours/latent_traversal_params_eps3/result_images7.pth')

In [ ]:
all_images = torch.stack([cf_image[30:40],
                          torch.flipud(cf_image_ours[30:40]),
                          cf_image_sample_two[30:40],
                          torch.flipud(cf_image_ours_sample_two[30:40]),
                          cf_image_wall[40:50],
                          cf_image_ours_wall[40:50],
                          cf_image_sample_two_wall[40:50],
                          cf_image_ours_sample_two_wall[40:50],
                          cf_image_object[50:60],
                          cf_image_ours_object[50:60],
                          cf_image_sample_two_object[50:60],
                          cf_image_ours_sample_two_object[50:60]
                         ]).view(-1,20,3,64,64)

In [ ]:
attr_list = ['Floor hue                               ',
             'Wall hue                               ',
             'Object hue                               ','Floor hue','Object hue','Object hue']
algo = ['SeFa', 'SeFa + SRE']

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

SMALL_SIZE = 8
plt.rc('axes', titlesize=22, labelsize=20)
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams["figure.facecolor"] = 'w'
plt.rcParams["font.family"] = "Times New Roman"

fig = plt.figure(figsize=(30, 10))
gs = gridspec.GridSpec(6, 1, wspace=0.01, hspace=0.01)
ax = np.zeros(6, dtype=object)
count = 0
for i in range(6):
    for j in range(1):
        ax[count] = fig.add_subplot(gs[i, j])
        grid = torchvision.utils.make_grid(all_images[i],nrow=10, scale_each=True, normalize=True)
        ax[count].imshow(grid.permute(1, 2, 0).cpu().numpy())
        ax[count].grid(False)
        ax[count].set_xticks([])
        ax[count].set_yticks([])
        count = count + 1
#         ax[i].title.set_text(attr_list[i])
# ax[0].set_ylabel(attr_list[0], rotation=90,fontsize=30)
# ax[1].set_ylabel(attr_list[1], rotation=90,fontsize=30)
# ax[2].set_ylabel(attr_list[2], rotation=90,fontsize=30)


gs.tight_layout(fig)
plt.savefig('latent_traversal_shapes3d_appendix_1.pdf', bbox_inches = 'tight')
